In [1]:
import os
import psutil
import numpy as np
from typing import Tuple
import mindspore as ms
from mindspore import nn, Tensor, ops

# 查询当前进程的内存，单位为MB
def get_cpu_memory(pid=os.getpid()):
    p = psutil.Process(pid)
    return p.memory_info().rss / (1024 * 1024)

class TestNet(nn.Cell):
    def __init__(self):
        super(TestNet, self).__init__()
        self._multinomial = ops.Multinomial()

    def construct(self, x):
        # 使用ops.multinomial函数式调用，存在泄露
        action = ops.multinomial(x, 1)
        # action = self._multinomial(x, 1)
        return action

def test_mem_leak():
    bs = 128
    c = 1024
    net = TestNet()
    x = Tensor(np.random.rand(bs, c), ms.float32)
    # warmup
    for i in range(50):
        y = net(x)
        y_arr = y.asnumpy()

    mem0 = get_cpu_memory()
    mem1 = 0
    for i in range(2000):
        y = net(x)
        y_arr = y.asnumpy()
        del y
        del y_arr
        mem1 = get_cpu_memory()     
        if (i+1) % 100 == 0:
            print('Step {}, cpu_mem {}MB -> {}MB'.format(i, mem0, mem1))

    print('All done: cpu_mem {}MB -> {}MB'.format(mem0, mem1))

if __name__ == '__main__':
    ms.context.set_context(device_target='GPU', mode=ms.context.PYNATIVE_MODE)
    # ms.context.set_context(device_target='GPU', mode=ms.context.GRAPH_MODE)

    test_mem_leak()

Step 99, cpu_mem 2427.96875MB -> 2428.21875MB
Step 199, cpu_mem 2427.96875MB -> 2428.30078125MB
Step 299, cpu_mem 2427.96875MB -> 2428.7890625MB
Step 399, cpu_mem 2427.96875MB -> 2429.05078125MB
Step 499, cpu_mem 2427.96875MB -> 2429.31640625MB
Step 599, cpu_mem 2427.96875MB -> 2429.3203125MB
Step 699, cpu_mem 2427.96875MB -> 2429.84375MB
Step 799, cpu_mem 2427.96875MB -> 2430.10546875MB
Step 899, cpu_mem 2427.96875MB -> 2430.3671875MB
Step 999, cpu_mem 2427.96875MB -> 2430.62890625MB
Step 1099, cpu_mem 2427.96875MB -> 2430.890625MB
Step 1199, cpu_mem 2427.96875MB -> 2431.1484375MB
Step 1299, cpu_mem 2427.96875MB -> 2432.73828125MB
Step 1399, cpu_mem 2427.96875MB -> 2434.80078125MB
Step 1499, cpu_mem 2427.96875MB -> 2437.12109375MB
Step 1599, cpu_mem 2427.96875MB -> 2439.69921875MB
Step 1699, cpu_mem 2427.96875MB -> 2442.01953125MB
Step 1799, cpu_mem 2427.96875MB -> 2444.33984375MB
Step 1899, cpu_mem 2427.96875MB -> 2446.14453125MB
Step 1999, cpu_mem 2427.96875MB -> 2448.72265625MB
All

In [11]:
from mindspore.dataset import MnistDataset, GeneratorDataset
# # since a generator instance can be only iterated once, we need to wrap it by lambda to generate multiple instances
dataset = GeneratorDataset(source=lambda: my_generator(3, 6), column_names=["data"])

# for d in dataset:
#     print(d)

# My_generator = my_generator(3, 6)
# dataset = GeneratorDataset(source=My_generator, column_names=["data"])

for i in range(5):
    for d in dataset:
        print(d)



[Tensor(shape=[], dtype=Int64, value= 3)]
[Tensor(shape=[], dtype=Int64, value= 4)]
[Tensor(shape=[], dtype=Int64, value= 5)]
[Tensor(shape=[], dtype=Int64, value= 3)]
[Tensor(shape=[], dtype=Int64, value= 4)]
[Tensor(shape=[], dtype=Int64, value= 5)]
[Tensor(shape=[], dtype=Int64, value= 3)]
[Tensor(shape=[], dtype=Int64, value= 4)]
[Tensor(shape=[], dtype=Int64, value= 5)]
[Tensor(shape=[], dtype=Int64, value= 3)]
[Tensor(shape=[], dtype=Int64, value= 4)]
[Tensor(shape=[], dtype=Int64, value= 5)]
[Tensor(shape=[], dtype=Int64, value= 3)]
[Tensor(shape=[], dtype=Int64, value= 4)]
[Tensor(shape=[], dtype=Int64, value= 5)]


In [25]:
import mindspore.dataset as ds
dataset = ds.GeneratorDataset([i for i in range(10)], "column1").batch(batch_size=2)
iterator = dataset.create_dict_iterator()
for item in iterator:
    # item is a list
    print(type(item),item,type(item["column1"]),item["column1"][1])
    break



<class 'dict'> {'column1': Tensor(shape=[2], dtype=Int64, value= [6, 8])} <class 'mindspore.common.tensor.Tensor'> 8


In [27]:
import mindspore.nn as nn
class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512, weight_init="normal", bias_init="zeros"),
            nn.ReLU(),
            nn.Dense(512, 512, weight_init="normal", bias_init="zeros"),
            nn.ReLU(),
            nn.Dense(512, 10, weight_init="normal", bias_init="zeros")
        )

    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

model = Network()

In [30]:
print(f"Model structure: {model}\n\n")

for name, param in model.parameters_and_names():
    print(f"Layer: {name}\nSize: {param.shape}\nValues : {param} \n")


Model structure: Network<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): ReLU<>
    (4): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >


Layer: dense_relu_sequential.0.weight
Size: (512, 784)
Values : Parameter (name=dense_relu_sequential.0.weight, shape=(512, 784), dtype=Float32, requires_grad=True) 

Layer: dense_relu_sequential.0.bias
Size: (512,)
Values : Parameter (name=dense_relu_sequential.0.bias, shape=(512,), dtype=Float32, requires_grad=True) 

Layer: dense_relu_sequential.2.weight
Size: (512, 512)
Values : Parameter (name=dense_relu_sequential.2.weight, shape=(512, 512), dtype=Float32, requires_grad=True) 

Layer: dense_relu_sequential.2.bias
Size: (512,)
Values : Parameter (name=dense_relu_sequential.2.bias, shape=(512,), dtype=Float32, requires_grad=True) 

Layer: d

In [37]:
import numpy as np
import mindspore as ms
import mindspore.nn as nn

ms.set_context(mode=ms.PYNATIVE_MODE)

def forward_pre_hook_fn(cell_id, inputs):
    print("forward inputs: ", inputs)
    return inputs

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.relu = nn.ReLU()
        self.handle = self.relu.register_forward_pre_hook(forward_pre_hook_fn)

    def construct(self, x, y):
        x = 2 * x + y
        x = self.relu(x)
        return x

net = Net()
grad_net = ms.grad(net, grad_position=(0, 1))

x = ms.Tensor(np.ones([1]).astype(np.float32))
y = ms.Tensor(np.ones([1]).astype(np.float32))

output = net(x, y)
print(output)
gradient = grad_net(x, y)
print(gradient)
net.handle.remove()
gradient = grad_net(x, y)
print(gradient)


forward inputs:  (Tensor(shape=[1], dtype=Float32, value= [ 3.00000000e+00]),)
[3.]
forward inputs:  (Tensor(shape=[1], dtype=Float32, value= [ 3.00000000e+00]),)
(Tensor(shape=[1], dtype=Float32, value= [ 2.00000000e+00]), Tensor(shape=[1], dtype=Float32, value= [ 1.00000000e+00]))
(Tensor(shape=[1], dtype=Float32, value= [ 2.00000000e+00]), Tensor(shape=[1], dtype=Float32, value= [ 1.00000000e+00]))
